In [2]:
# import packages
from statsmodels.formula.api import ols
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline
# allow more collumns and rows to be seen
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn import linear_model
from sklearn.feature_selection import RFECV
from sklearn import metrics
import scipy.stats as stats
from scipy import stats
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [3]:
df = pd.read_csv('decoded_mushrooms.csv', index_col = 0)
df2 = pd.read_csv('decoded_num_mushrooms.csv', index_col = 0)

In [4]:
df.head(3)
df2.head(3)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2,1,1,2,1,2,1,1,2,1,1,2,1,1,1,1,1,1,1,1,2,2,2
2,2,2,1,3,1,3,1,1,2,2,1,2,1,1,1,1,1,1,1,1,2,2,3


In [55]:
y = df2['class']
# removed gill-color due to nans for now
X = df2.drop(['class'] , axis = 1)

In [56]:
# Split X and y with even class distributions
# stratify to create randomness in your data but one that also has the same propoertoins across train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)

In [41]:
#Scale the data
ss = StandardScaler()
ss.fit(X_train)
X_train = pd.DataFrame(ss.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(ss.transform(X_test), columns=X_test.columns)

In [42]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

C:\Users\akams\anaconda3\envs\learn-env\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1.0

In [43]:
#Now with cross validation search
val = cross_val_score(RandomForestClassifier(random_state=42),X_train,y_train,cv=5)
val.mean()

C:\Users\akams\anaconda3\envs\learn-env\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\akams\anaconda3\envs\learn-env\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\akams\anaconda3\envs\learn-env\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\akams\anaconda3\envs\learn-env\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\akams\a

1.0

Try without spore-print

In [35]:
y = df2['edible']
# removed gill-color due to nans for now
Z = df2[['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'population', 'habitat']]

In [36]:
# Split X and y with even class distributions
# stratify to create randomness in your data but one that also has the same propoertoins across train and test
Z_train, Z_test, y_train, y_test = train_test_split(Z, y, test_size=.2, random_state=42, stratify=y)

In [37]:
#Scale the data
ss = StandardScaler()
# Fit scaler to training set
ss.fit(Z_train)
# transform test and train data
Z_train = pd.DataFrame(ss.transform(Z_train), columns=Z_train.columns)
Z_test = pd.DataFrame(ss.transform(Z_test), columns=Z_test.columns)

In [38]:
rf = RandomForestClassifier(random_state=42)
rf.fit(Z_train, y_train)
rf.score(Z_test, y_test)

C:\Users\akams\anaconda3\envs\learn-env\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1.0

RF with Grid Search

In [5]:
df3 = pd.read_csv('mushroom_ft_engineer.csv', index_col = 0)

In [34]:
df3.isna().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
location                    0
smell                       0
green                       0
spore_color                 0
no_rings                    0
human_interference          0
smell_quality               0
c_molybdites                0
dtype: int64

In [47]:
df3.columns

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat', 'location',
       'smell', 'green', 'spore_color', 'no_rings', 'human_interference',
       'smell_quality', 'c_molybdites'],
      dtype='object')

In [6]:
# define x and y 
y = df3['class']
X = df3[['cap-shape', 'cap-surface', 'cap-color', 'bruises',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color',
        'population', 'no_rings', 'human_interference',
       'smell_quality']]
# Split X and y with even class distributions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)

In [8]:
rf_param_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 2, 4, 6, 10],
    "max_features": [None,4,5,6,9,10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf" : [1, 2, 3, 5, 6],
    "n_estimators" : [10, 30, 100, 500]
}


rf_grid = RandomForestClassifier(random_state=42)
gridsearch = GridSearchCV(rf_grid, rf_param_grid, cv=3, return_train_score=True, n_jobs=-1, verbose=-1)

gridsearch.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 3512 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done 4592 tasks      | elapsed: 19.4min
[Parallel(n_jobs=-1)]: Done 5816 tasks      | elapsed: 24.6min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed: 30.6min
[Parallel(n_jobs=-1)]: Done 8696 tasks      | elapsed: 36.7min
[Parallel(n_jobs=-1)]: Done 10352 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 10800 out of 10800 | elapsed: 45.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'cr

In [9]:
print("Testing Accuracy: {:.4}%".format(gridsearch.best_score_ * 100))
print("")
print("Optimal Parameters: {}".format(gridsearch.best_params_))

# Actual model object fit with those best parameters
# Shows default parameters that we did not specify
print("Best Model: {}".format(gridsearch.best_estimator_))

Testing Accuracy: 100.0%

Optimal Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 4, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}
Best Model: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=4, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


In [10]:
gridsearch.score(X_test, y_test)

1.0

Decision Tree

In [59]:
# define x and y 
y = df3['class']
X = df3.drop(['class'] , axis = 1)
# Split X and y with even class distributions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)

In [60]:
from sklearn.tree import DecisionTreeClassifier
ctree = DecisionTreeClassifier(max_depth = 10)
dtree = ctree.fit(X_train,y_train)

In [68]:
clasPred = ctree.predict(X_test)

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, clasPred)

1.0